<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [2]:
import pandas as pd
import psycopg2

In [7]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [307]:
# текст запроса
query_3_1 = f'''select count(*) from vacancies'''

In [304]:
# результат запроса
# 49197
df = pd.read_sql_query(query_3_1, connection)
df

,count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [ ]:
# текст запроса
query_3_2 = f'''select count(*) from employers'''

In [306]:
# результат запроса
# 23501
df = pd.read_sql_query(query_3_2, connection)
df

,count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [ ]:
# текст запроса
query_3_3 = f'''select count(*) from areas'''

In [305]:
# результат запроса
# 1362
df = pd.read_sql_query(query_3_3, connection)
df

,count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [ ]:
# текст запроса
query_3_4 = f'''select count(*) from industries'''

In [308]:
# результат запроса
# 294
df = pd.read_sql_query(query_3_4, connection)
df

,count
0,294


***

In [ ]:
# выводы по предварительному анализу данных
Предварительный анализ данных показал, что База Данных вакансий состоит из 5 таблиц, занимает около 22МБ и содержит информацию о:
- 49197 вакансиях (таблица vacancies),
- 23501 работодателе (таблица employers),
- 1362 регионах (таблица areas),
- 294 сферах деятельности (таблица industries).

В качестве хранилища данных используется СУБД PostgreSQL 11.3.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [ ]:
# текст запроса
query_4_1 = f'''-- Выборка
select (select name from areas where id = v.area_id) area_name,
       count(*) vacancies_cnt
  from vacancies v
  group by v.area_id
  order by count(*) desc
'''

In [309]:
# результат запроса
df = pd.read_sql_query(query_4_1, connection)
df

,area_name,vacancies_cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
...,...,...
764,Городец,1
765,Шлиссельбург,1
766,Мурмаши,1
767,Семилуки,1


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [ ]:
# текст запроса
query_4_2 = f'''-- Выборка
select count(*) vacancies_with_salary
  from vacancies
  where salary_from is not null or
        salary_to is not null
'''

In [310]:
# результат запроса
# 24073
df = pd.read_sql_query(query_4_2, connection)
df

,vacancies_with_salary
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [ ]:
# текст запроса
query_4_3 = f'''-- Выборка
select avg(salary_from)::int salary_from_avg,
       avg(salary_to)::int salary_to_avg
  from vacancies
'''

In [311]:
# результат запроса
# 71065 - 110537
df = pd.read_sql_query(query_4_3, connection)
df

,salary_from_avg,salary_to_avg
0,71065,110537


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [ ]:
# текст запроса
query_4_4 = f'''-- Выборка
select schedule,
       employment,
       count(*) vacancies_cnt
  from vacancies
  group by schedule,
           employment
  order by count(*) desc
'''

In [312]:
# результат запроса
df = pd.read_sql_query(query_4_4, connection)
df

,schedule,employment,vacancies_cnt
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [ ]:
# текст запроса
query_4_5 = f'''-- Выборка
select experience,
       count(*) vacancies_cnt
  from vacancies
  group by experience
  order by count(*)
'''

In [313]:
# результат запроса
df = pd.read_sql_query(query_4_5, connection)
df

,experience,vacancies_cnt
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

In [ ]:
# выводы по детальному анализу вакансий
Детальный анализ данных показал:
Детальный анализ вакансий включал метрики:
1. Количество вакансий в разрезе регионов
Наибольшее число вакансий находится в Москве. Второе место по количеству вакансий у Санкт-Петербурга.
При соотношение вакансий Москвы и Санкт-Петербурга можно сравнить с соотношением вакансий в этих городах - больше население - больше вакансий.
При этом данная тенденция не сохраняется для третьего по величине населения и количеству вакансий города России - Новосибирска. При значительно меньшем населении, количество вакансий снижается в меньшей степени.
2. Количество вакансий с указанием указанием заработной платы
Работадатели указывают уровень заработной платы только в половине случаев (из 49197 вакансий зарплата указана в 24073)
3. Среднее значение нижней и верхней границы зарплатной вилки
Данные значения получены без учета опыта работы соискателей и могут быть сильно искажены, например, средний уровень нижней границы может оказаться выше верхней границы для стажера.
4. Количество вакансий для каждого сочетания типа рабочего графика и типа трудоустройства
Согласно полученным результатам работодатели заинтересованы в найме сотрудников на полную занятость для работы на территории работодателя.
Второй по популярности вакансии с полной занятостью и удаленой работе. В БД отсутствует информация о дате появления вакансии, что затрудняет понимание, есть ли тенденция к увеличению появлению удаленной работы или нет (без даты такую оценку можно сделать по ID вакансии если это автоинкрементное поле).
Достточно трудно найти работу для стажировки или с частичной занятостью.
5. Количество вакансий в зависимости от опыта работы
Наиболее востребованы сотрудники с опытом работы от 1 года до 3-х лет. Такие сотрудники обладают некоторым опытом, способны самостоятельно выполнять задания и при этом не требуют максимального оклада.
Нименьшее количество предложений для соискателей с большим опытом работы. Возможно это связано с тем, что таким людям нужно больше платить и возраст. Руководящие должности.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [ ]:
# текст запроса
employers_places = '1, 5'
query_5_1 = f'''-- Выборка
select *
  from (
         select employer_name,
                --vacancies_cnt,
                dense_rank() over (order by vacancies_cnt desc) employer_rank
           from (
                  select (select name from employers where id = v.employer_id) employer_name,
                         count(*) vacancies_cnt
                    from vacancies v
                    group by v.employer_id
                ) t
       ) t2
  where employer_rank in ({employers_places})
'''

In [314]:
# результат запроса
df = pd.read_sql_query(query_5_1, connection)
df

,employer_name,employer_rank
0,Яндекс,1
1,Газпром нефть,5


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [ ]:
# текст запроса
query_5_2 = f'''-- Выборка
select --a.id area_id,
       a.name area_name,
       e.employers_cnt,
       v.vacancies_cnt
  from areas a
       left join (
         select area_id,
                count(*) vacancies_cnt
           from vacancies
           group by area_id
       ) v on v.area_id = a.id
       left join (
         select area,
                count(*) employers_cnt
           from employers
           group by area
       ) e on e.area = a.id
  where v.area_id is null
  order by e.employers_cnt desc nulls last
  limit 1
'''

In [315]:
# результат запроса
df = pd.read_sql_query(query_5_2, connection)
df

,area_name,employers_cnt,vacancies_cnt
0,Россия,410,None


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [316]:
# текст запроса
query_5_3 = f'''-- Выборка
select employer_id,
       count(distinct area_id) vacancy_areas_cnt
  from vacancies
  group by employer_id
  order by count(distinct area_id) desc
'''

In [317]:
# результат запроса
df = pd.read_sql_query(query_5_3, connection)
df

,employer_id,vacancy_areas_cnt
0,1740,181
1,2748,152
2,5724811,116
3,5130287,88
4,3682876,71
...,...,...
14901,810278,1
14902,810313,1
14903,810551,1
14904,810688,1


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [ ]:
# текст запроса
query_5_4 = f'''-- Выборка
select count(*) employers_without_industry
  from employers e
       left join employers_industries ei on ei.employer_id = e.id
  where ei.employer_id is null
'''

In [318]:
# результат запроса
# 8419
df = pd.read_sql_query(query_5_4, connection)
df

,employers_without_industry
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [ ]:
# текст запроса
employer_industries_cnt = 4
query_5_5 = f'''-- Выборка
select ei.employer_id,
       (select name from employers where id = ei.employer_id) employer_name
  from employers_industries ei
  group by ei.employer_id
  having count(*) = {employer_industries_cnt}
  order by employer_name
  offset 2 rows
  fetch next 1 row only
'''

In [319]:
# результат запроса
# 2ГИС
df = pd.read_sql_query(query_5_5, connection)
df

,employer_id,employer_name
0,64174,2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [ ]:
# текст запроса
industry_name = 'Разработка программного обеспечения'
query_5_6 = f'''-- Выборка
select count(*) soft_dev_employers_cnt
  from employers_industries ei
  where industry_id = (select id from industries where name = '{industry_name}')
'''

In [320]:
# результат запроса
# 3553
df = pd.read_sql_query(query_5_6, connection)
df

,soft_dev_employers_cnt
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [321]:
# код для получения списка городов-милионников
import requests
from bs4 import BeautifulSoup

url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'
response = requests.get(url)
page = BeautifulSoup(response.text, 'html.parser')
cities = [city.find('a').text for city in page.find('table', class_='standard').find_all('td')[1::5]]

In [322]:
# текст запроса
employer_name = 'Яндекс'
query_5_7 = f'''-- Выборка количества вакансии заданной компании в разрезе регионов-миллионников. 
select coalesce(a.name, 'Total') area_name,
       count(*) vacancies_cnt
  from vacancies v
       join areas a on v.area_id = a.id
  where v.employer_id = (select id from employers where name = '{employer_name}') and
        a.name = any(array[{cities}])
  group by grouping sets ((a.name), ())
  order by vacancies_cnt
'''

In [323]:
# результат запроса
df = pd.read_sql_query(query_5_7, connection)
df

,area_name,vacancies_cnt
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Самара,26
8,Уфа,26
9,Краснодар,30


***

In [ ]:
# выводы по анализу работодателей
По вакансиям среди работодателей с большим отрывом лидирует "Яндекс" - 1933 вакансии. У второй компании "Ростелеком" вакансий в 4 раза меньше - 491.
Эти две компании имеют наибольшее представительство и в регионах - 181 и 152 соответственно.
В большинстве организаций небольшое число вакансий. Только у первых 25 компаний количество вакансий вакансий превышает 100.

Лидерами по количеству работодателей и вакнсий являются Москва и Санкт-Петербург. Только в этих городах количество работодателей сравнимо с числом вакансий.
Это может быть связано с тем, что работодатели в этих городах осуществляют подбор персонала в свои региональные филиалы.

У трети работодателей отсутствуют сферы деятельности.
Наиболее востребованными сферами деятельности являются:
    "Разработка программного обеспечения",
    "Системная интеграция,  автоматизации технологических и бизнес-процессов предприятия, ИТ-консалтинг",
    "Интернет-компания (поисковики, платежные системы, соц.сети, информационно-познавательные и развлекательные ресурсы, продвижение сайтов и прочее)".
Их доля составляет 15%, 12% и 7% соответственно.

Компания "Яндекс" представлена во всех городах-миллионерах России достаточно равномерно (количеством вакансий выделяется только Москва).
Отсутствует зависимость вакансий от количества населения. 

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [ ]:
# текст запроса
query_6_1 = f'''-- Выборка
select count(*) vacancies_cnt
  from vacancies
  where upper(name) like '%DATA%' or
        upper(name) like '%ДАНН%'
'''

In [324]:
# результат запроса
# 1771
df = pd.read_sql_query(query_6_1, connection)
df

,vacancies_cnt
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [325]:
# текст запроса
query_6_2 = f'''-- Выборка
select count(*) vacancies_cnt
  from vacancies
  where (
          upper(name) like '%DATA SCIENTIST%' or
          upper(name) like '%DATA SCIENCE%' or
          upper(name) like '%ИССЛЕДОВАТЕЛЬ ДАННЫХ%' or
          --name like '%ML%' or -- Только в верхнем регистре. В противном случае может совпасть с TeamLead. todo: заменить регулярным выражением, чтобы убрать строки для HTML и QML
          name ~ '(^|\W)[AQ]?ML($|\W)' or -- [AQ]? не влияет на результат запроса. влияет на 6.3
          name ~ '(^|\W)ML(E|Ops)($|\W)' or -- не влияет на результат запроса. влияет на 6.3
          upper(name) like '%MACHINE LEARNING%' or
          upper(name) like '%МАШИНН%ОБУЧЕН%'
        ) and
        --upper(name) not like '%HTML%' and
        --upper(name) not like '%TEAMLEAD%' and
        --upper(name) not like '%QML%' and
        (
          upper(name) like '%JUNIOR%' or
          experience = 'Нет опыта' or
          employment = 'Стажировка'
        )
'''

In [326]:
# результат запроса
# 51
df = pd.read_sql_query(query_6_2, connection)
df

,vacancies_cnt
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [327]:
# текст запроса
query_6_3 = f'''-- Выборка
select count(distinct id) vacancies_cnt
  from (
         select id,
                unnest(string_to_array(key_skills, chr(9))) skill
           from vacancies
           where (
                   upper(name) like '%DATA SCIENTIST%' or
                   upper(name) like '%DATA SCIENCE%' or
                   upper(name) like '%ИССЛЕДОВАТЕЛЬ ДАННЫХ%' or
                   --name like '%ML%' or
                   name ~ '(^|\W)[AQ]?ML($|\W)' or -- для совпадения с ответом задания 6.3 должны отбираться строки вакансий "C++ разработчик (QML)", "Java Разработчик (направление AML)", "Бизнес-аналитик (на оптимизацию опер.инструментов AML)" с id in (52995746, 48838707, 54753924)
                   name ~ '(^|\W)ML(E|Ops)($|\W)' or -- для совпадения с ответом задания 6.3 должны отбираться строки вакансий "Middle MLE (Big Data Department)", "MLOps Engineer в области рекомендательных систем" с id in (55004712, 55004713, 55138761)
                   upper(name) like '%MACHINE LEARNING%' or
                   upper(name) like '%МАШИНН%ОБУЧЕН%'
                 ) --and
                 --upper(name) not like '%HTML%' --and
                 --upper(name) not like '%TEAMLEAD%' --and
                 --upper(name) not like '%QML%' -- для совпадения с ответом задания 6.3 должна отбираться строка вакансии с id=52995746 (C++ разработчик (QML)) не относящаяся к дата-сайентистам
       ) t
  where upper(skill) like '%SQL%' or -- для совпадения с ответом задания 6.3 должны отбираться навыки PL/SQL, MYSQL, MS SQL при том, что есть отдельный навык SQL
        upper(skill) like '%POSTGRES%' -- Все строки для фильтра по POSTGRES попадают по фильту SQL, т.к. везде значение PostgreSQL
'''

In [328]:
# результат запроса
# 201
df = pd.read_sql_query(query_6_3, connection)
df

,vacancies_cnt
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [329]:
# текст запроса
query_6_4 = f'''-- Выборка
select count(distinct id) vacancies_cnt
  from (
         select id,
                unnest(string_to_array(key_skills, chr(9))) skill
           from vacancies
           where (
                   upper(name) like '%DATA SCIENTIST%' or
                   upper(name) like '%DATA SCIENCE%' or
                   upper(name) like '%ИССЛЕДОВАТЕЛЬ ДАННЫХ%' or
                   name like '%ML%' or
                   upper(name) like '%MACHINE LEARNING%' or
                   upper(name) like '%МАШИНН%ОБУЧЕН%'
                 ) and
                 upper(name) not like '%HTML%' and
                 --upper(name) not like '%TEAMLEAD%' --and
                 upper(name) not like '%QML%' -- не влияет на результат запроса
       ) t
  where upper(skill) like '%PYTHON%'
'''

In [330]:
# результат запроса
# 351
df = pd.read_sql_query(query_6_4, connection)
df

,vacancies_cnt
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [331]:
# текст запроса
query_6_5 = f'''-- Выборка
select round(avg(cardinality(string_to_array(key_skills, chr(9)))), 2) ds_skills_avg
  from vacancies
  where (
          upper(name) like '%DATA SCIENTIST%' or
          upper(name) like '%DATA SCIENCE%' or
          upper(name) like '%ИССЛЕДОВАТЕЛЬ ДАННЫХ%' or
          name like '%ML%' or
          upper(name) like '%MACHINE LEARNING%' or
          upper(name) like '%МАШИНН%ОБУЧЕН%'
        ) and
        upper(name) not like '%HTML%' and
        --upper(name) not like '%TEAMLEAD%' and
        --upper(name) not like '%QML%' and -- для совпадения с ответом задания 6.5 строка должна быть закомментирована
        key_skills is not null
'''

In [332]:
# результат запроса
#6.41
df = pd.read_sql_query(query_6_5, connection)
df

,ds_skills_avg
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [5]:
# текст запроса
experience = 'От 3 до 6 лет'
query_6_6 = f'''-- Выборка
select experience,
       avg(
         case
           when salary_from is null or salary_to is null then coalesce(salary_from, 0) + coalesce(salary_to, 0)
           else (salary_from + salary_to)/2.0
         end
       )::int salary
  from vacancies
  where (
          upper(name) like '%DATA SCIENTIST%' or
          upper(name) like '%DATA SCIENCE%' or
          upper(name) like '%ИССЛЕДОВАТЕЛЬ ДАННЫХ%' or
          name like '%ML%' or
          upper(name) like '%MACHINE LEARNING%' or
          upper(name) like '%МАШИНН%ОБУЧЕН%'
        ) and
        upper(name) not like '%HTML%' and
        upper(name) not like '%TEAMLEAD%' and
        upper(name) not like '%QML%' and -- не влияет на результат запроса для опыта 'От 3 до 6 лет', но влияет для опыта 'От 1 года до 3 лет'
        (salary_from is not null or salary_to is not null) and
        experience = '{experience}'
  group by experience
'''

In [8]:
# результат запроса
# 243115
df = pd.read_sql_query(query_6_6, connection)
connection.close()
df

,experience,salary
0,От 3 до 6 лет,243115


***

In [ ]:
# выводы по предметному анализу
В предметном анализе рассматривались вакансии, относящиеся к работе с данными.
Число таких предложений небольшое - около 3.5% (1771) относительно общего числа вакансий (49197).
Из этих 3.5% для начинающего специалиста подойдет 2.8% (51 вакансия), что ставит под сомнение целесообразность данных курсов.
Работодатели предпочитают молодых специалистов с небольшим, но не нулевым опытом. Специалисты с большим опытом менее востребованы.

# Общий вывод по проекту

In [ ]:
# подведем итог исследования, обобщите выводы
# здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, продумать варианты продолжения исследования
При работе над проектом получены практические навыки в написании запросов SQL с учетом особенностей БД PostgreSQL.
Выполнен предварительный анализ данных, детальный анализ вакансий, анализ работодателей и предметный анализ.
Дополнительно можно провести анализ опыта работы с каким наиболее востребованы специалисты, возраста специалистов, как изменяется со временем соотношение вакансий для удаленной работы и работы в офисе.